In [18]:
from splinter import Browser
from bs4 import BeautifulSoup
import pymongo
import requests
import pandas as pd
import time

In [6]:
mars_tweet_url="https://twitter.com/marswxreport?lang=en"
#soup = BeautifulSoup(mars_tweet_url, 'html.parser')

In [7]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [19]:

def init_browser():
    chrome_location = "chromedriver.exe"    
    executable_path = {"executable_path":chrome_location}
    
    return Browser("chrome", **executable_path, headless=False)

def get_soup_object(url):
    browser = init_browser()
    browser.visit(url)
    soup = BeautifulSoup(browser.html,"html.parser")
    return soup

def get_MARS_temperature():
    twitter_report_url = "https://twitter.com/marswxreport?lang=en"
    temp_soup = get_soup_object(twitter_report_url)
    return temp_soup.find("ol",{"id":"stream-items-id"}).find("li").find("p").text



def get_MARS_facts():
    df = pd.read_html("https://space-facts.com/mars/")[0]
    df = df.rename(columns={0:"Description",1:"Value"})
    return df.to_dict()



def get_MARS_hemisphere_data():
    browser = init_browser()
    hemispheres_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
    browser.visit(hemispheres_url)
    hms_soup = BeautifulSoup(browser.html,"html.parser")
    items = hms_soup.find("div",{"id":"product-section"}).find_all("div",{"class":"item"})

    hemisphere_data = []

    for item in items:
        img_main_url = "https://astrogeology.usgs.gov"+item.find("a")["href"]
        img_title = item.find("div",{"class":"description"}).find("a").find("h3").text
        browser.visit(img_main_url)
        time.sleep(1)
        img_soup = BeautifulSoup(browser.html,"html.parser")
        download_item = img_soup.find("div",{"class":"downloads"})
        hemisphere_item = {
            "title":img_title,
            "img_url": download_item.find("li").find("a")["href"]
        }
        hemisphere_data.append(hemisphere_item)
        
    return hemisphere_data



def scrape():    
    mars_facts = get_MARS_facts()
    mars_temp = get_MARS_temperature()
    mars_hm_data = get_MARS_hemisphere_data()
    return {        
        "facts":mars_facts,
        "temperature":mars_temp,
        "hemisphere_data":mars_hm_data
    }

scrape()

{'facts': {'Description': {0: 'Equatorial Diameter:',
   1: 'Polar Diameter:',
   2: 'Mass:',
   3: 'Moons:',
   4: 'Orbit Distance:',
   5: 'Orbit Period:',
   6: 'Surface Temperature:',
   7: 'First Record:',
   8: 'Recorded By:'},
  'Value': {0: '6,792 km',
   1: '6,752 km',
   2: '6.42 x 10^23 kg (10.7% Earth)',
   3: '2 (Phobos & Deimos)',
   4: '227,943,824 km (1.52 AU)',
   5: '687 days (1.9 years)',
   6: '-153 to 20 °C',
   7: '2nd millennium BC',
   8: 'Egyptian astronomers'}},
 'temperature': 'InSight sol 129 (2019-04-07) low -97.7ºC (-143.9ºF) high -16.7ºC (1.9ºF)\nwinds from the SW at 4.1 m/s (9.1 mph) gusting to 11.9 m/s (26.7 mph)\npressure at 7.30 hPapic.twitter.com/UbKB6hFeoI',
 'hemisphere_data': [{'title': 'Cerberus Hemisphere Enhanced',
   'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
  {'title': 'Schiaparelli Hemisphere Enhanced',
   'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking